In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import xarray as xr
import numdifftools as ndt

from scipy.optimize import minimize
from pyDOE import *
from scipy.linalg import cholesky
from scipy.interpolate import interp1d
from scipy import signal
from eofs.standard import Eof
from netCDF4 import Dataset
from scipy.interpolate import griddata
from mySSA import mySSA
from sklearn.decomposition import PCA
from statsmodels.tsa import stattools
from scipy.signal import butter, lfilter
from scipy.integrate import nquad, quad

from tsmoothie.utils_func import sim_seasonal_data
from tsmoothie.smoother import ConvolutionSmoother
from tsmoothie.bootstrap import BootstrappingWrapper

from sklearn.metrics import mean_squared_error

# FUNCTIONS

In [ ]:
def ukf_ndim(para):
     
    """
    Performs ukf (na-dimentions)

    Parameters:
    - para (list): parameters for the potential

    Returns:
    float: log likelihood
    """
    
    n = len(data)
    na = 1
    na2 = na*2
    H = np.ones((1, na))
    h = 0.001
    R = para[4]**2
    Q = h*para[5]**2
    transient = 50
    x_pre = [data[0]]
    p_pre = np.full((na, na), R)
    loc_lnlike = np.zeros(n)
    
    for k in range(2,n+1):
        
        for l in range(1,L+1):
            
            A = np.linalg.cholesky(na*p_pre)
            
            sig_p = np.hstack((x_pre + A, x_pre - A))
            
            if ukf_ndim.mode == "full":
                sig_p[0, :] = sig_p[0, :] - h * (4.0 * para[3] * sig_p[0, :] ** 3 + 3.0 
                                             * para[2] * sig_p[0, :] ** 2 + 2.0 * para[1] * sig_p[0, :] + para[0]
                                                + para[6] * benthic[k-1] + para[7] * sol[k-1])
            if ukf_ndim.mode == "no":
                sig_p[0, :] = sig_p[0, :] - h * (4.0 * para[3] * sig_p[0, :] ** 3 + 3.0 
                                             * para[2] * sig_p[0, :] ** 2 + 2.0 * para[1] * sig_p[0, :] + para[0])
                
            if ukf_ndim.mode == "benthic":
                sig_p[0, :] = sig_p[0, :] - h * (4.0 * para[3] * sig_p[0, :] ** 3 + 3.0 
                                             * para[2] * sig_p[0, :] ** 2 + 2.0 * para[1] * sig_p[0, :] + para[0]
                                                + para[6] * benthic[k-1])
            if ukf_ndim.mode == "sol":
                sig_p[0, :] = sig_p[0, :] - h * (4.0 * para[3] * sig_p[0, :] ** 3 + 3.0 
                                             * para[2] * sig_p[0, :] ** 2 + 2.0 * para[1] * sig_p[0, :] + para[0]
                                                + para[6] * sol[k-1])
            
            x_pre = np.mean(sig_p, axis=1)
            p_pre = np.zeros((na, na))
        
            for i in range(1,na2+1):
                p_pre += np.outer(sig_p[:, i-1] - x_pre, sig_p[:, i-1] - x_pre)
                
            p_pre = p_pre/na2+Q
            
        S = H @ p_pre @ H.T + R   
        K = p_pre @ H.T @ np.linalg.inv(S)
        resi = data[k-1] - H @ x_pre    
        loc_lnlike[k-1] = -(np.log(np.linalg.det(S)) + np.dot(resi.T, np.linalg.solve(S, resi))) / 2 
        x_pre = x_pre + K @ resi        
        p_pre = np.dot((np.eye(na) - np.dot(K, H)), p_pre) 
    
    return - (-(n-transient)/2*np.log(2*np.pi)+sum(loc_lnlike[(transient):n]))

In [ ]:
#def ukf_1dim(para, h):
def ukf_1dim(para):
    
    """
    Performs ukf (1-dimension)

    Parameters:
    - para (list): parameters for the potential
    - h (float): stepsize of ukf

    Returns:
    float: log likelihood
    """
    
    n = len(data)
    h = 0.001   
    R = para[4]**2 
    Q = h*para[5]**2 
    transient = 50 
    L = int(dT/h)

    x_pre = np.zeros(n)
    p_pre = np.zeros(n)
    
    x_pre[0] = data[0]
    
    if p_pre[0] == 0:
        p_pre[0] = R
    
    loc_lnlike = np.zeros(n)
    
    for k in range(1,n):
        
        x_pre[k] = x_pre[k-1]
        p_pre[k] = p_pre[k-1]
        
        for l in range(1, L+1): 
            A = np.sqrt(p_pre[k])
            sp = [x_pre[k]+A,x_pre[k]-A]
            
            if ukf_1dim.mode == "full":
                if ukf_1dim.hem=="north":
                    sp = [x - h*(4*para[3]*x**3 + 3*para[2]*x**2 + 2*para[1]*x + para[0] + para[6] * benthic[k] + para[7] * soln[k]) for x in sp]
                if ukf_1dim.hem=="south":
                    sp = [x - h*(4*para[3]*x**3 + 3*para[2]*x**2 + 2*para[1]*x + para[0] + para[6] * benthic[k] + para[7] * sols[k]) for x in sp]
            if ukf_1dim.mode == "no":
                sp = [x - h*(4*para[3]*x**3 + 3*para[2]*x**2 + 2*para[1]*x + para[0]) for x in sp]
            if ukf_1dim.mode == "benthic":
                sp = [x - h*(4*para[3]*x**3 + 3*para[2]*x**2 + 2*para[1]*x + para[0] + para[6] * benthic[k]) for x in sp]
            if ukf_1dim.mode == "sol":
                if ukf_1dim.hem=="north":
                    sp = [x - h*(4*para[3]*x**3 + 3*para[2]*x**2 + 2*para[1]*x + para[0] + para[6] * soln[k]) for x in sp]
                if ukf_1dim.hem=="south":
                    sp = [x - h*(4*para[3]*x**3 + 3*para[2]*x**2 + 2*para[1]*x + para[0] + para[6] * sols[k]) for x in sp]
                    
            x_pre[k] = np.mean(sp)
            p_pre[k] = np.var(sp) + Q
           
        S = p_pre[k]+R
        K = p_pre[k]/S
        resi = data[k]-x_pre[k]
        loc_lnlike[k] = -(np.log(S)+resi**2/S)/2
        x_pre[k] = x_pre[k] + K * resi
        p_pre[k] = (1-K)*p_pre[k]
               
    r = - (-(n-transient)/2*np.log(2*np.pi)+sum(loc_lnlike[(transient):n]))
    if r is None or isinstance(r, float) and math.isnan(r):
        return -100000000
    else:
        return r

In [ ]:
def data_cut(data, start, end):
    
    """
    cut data to a specific time period. 

    Parameters:
    - data (list): data that wants to be shorten in time
    - start (int): start date, must be larger than minimum time of data
    - end (int):   end date, must be shorten than maximimum time of data

    Returns:
    list: [cut time, cut data]
    """
    
    start_index = next((i for i, value in enumerate(t) if value >= start), None)
    end_index = next((i for i, value in reversed(list(enumerate(t))) if value <= end), None)

    if start_index is not None and end_index is not None:
        t_ = t[start_index:end_index + 1]
        data_ = data[start_index:end_index + 1]
        return t_, data_
    else:
        return None, None

In [ ]:
def SSA(data, K, suspected_seasonality, start, end):
    """
    performs singular component analysis between data1 and data2
    
    Parameters:
    - data1 (list): time series data1
    - data2 (list): time series data2
    
    Returns:
    [0]: 
    [1]: 
    """
    
    ts = pd.DataFrame({'y': data})
    
    start_year = 2000
    num_rows = len(data) 

    date_index = pd.date_range(start=f"{start_year}-01-01", periods=num_rows, freq='H')

    ts.index = date_index
    ssa = mySSA(ts)

    #K = int(10 / dT)
    K = K
    #suspected_seasonality = 5
    suspected_seasonality = suspected_seasonality
    
    #Embed the time series by forming a Hankel matrix of lagged window (length K) vectors
    ssa.embed(embedding_dimension=K, suspected_frequency=suspected_seasonality, verbose=False)
    #Decompose the embedded time series via Singular Value Decomposition
    ssa.decompose(verbose=False)
    
    streams = [i for i in range(start,end)]
    reconstructed = ssa.view_reconstruction(*[ssa.Xs[i] for i in streams], 
                                          names=streams, return_df=True, plot=False)


    return reconstructed["Reconstruction"]

In [ ]:
def potential(x):
    """
    calculates the potential for given parameters
    
    Parameters:
    - x (list): parameters of potential
    
    Returns:
    list: 
    """
    return (p[3]*x**4 + p[2]*x**3 + p[1]*x**2 + p[0]*x)

def potential_fit(x):
    """
    performs singular component analysis between data1 and data2
    
    Parameters:
    - data1 (list): time series data1
    - data2 (list): time series data2
    
    Returns:
    [0]: 
    [1]: 
    """
    return (fit.x[3]*x**4 + fit.x[2]*x**3 + fit.x[1]*x**2 + fit.x[0]*x)

In [ ]:
def plot_potential(x, p):
    return (p[3]*x**4 + p[2]*x**3 + p[1]*x**2 + p[0]*x)

def plot_error(param, hess, label):
    #line = plt.plot(xx, [plot_potential(x, param) for x in xx], label = str(label))
    param_samples = np.random.multivariate_normal(param, np.diag(hess), num_samples)
    potential_samples = np.array([plot_potential(xx, p) for p in param_samples])
    for i in range(len(potential_samples)):
        plt.plot(xx, potential_samples[i], c="k", alpha=0.02)
    #quantiles = np.percentile(potential_samples, [40, 100-40], axis=0)
    #plt.fill_between(xx[:,0], quantiles[0][:,0], quantiles[1][:,0], alpha=0.8)

In [ ]:
def plot_std(dx, num):
    curve = [potential_fit(x) for x in xx]
    for i in range(num):
        a = num
        dcurve = [np.sqrt((x**4*i/a*dx[3])**2+(x**3*i/a*dx[2])**2+(x**2*i/a*dx[1])**2+(x*i/a*dx[0])**2) for x in xx]
        plt.fill_between(xx, [a-b for a, b in zip(curve, dcurve)], [a+b for a, b in zip(curve, dcurve)],color = "k", alpha = 1/a, edgecolor = "none")
    
def plot_std_(dx, color="k"):
    curve = [potential_fit(x) for x in xx]
    dcurve = [np.sqrt((x**4*dx[3])**2+(x**3*dx[2])**2+(x**2*dx[1])**2+(x*dx[0])**2) for x in xx]
    plt.fill_between(xx, [a-b for a, b in zip(curve, dcurve)], [a+b for a, b in zip(curve, dcurve)],color = color, alpha = 0.6, edgecolor = "none")
    

In [ ]:
def lagged_corr(x,y):

    def ccf_values(series1, series2):
        p = series1
        q = series2
        p = (p - np.mean(p)) / (np.std(p) * len(p))
        q = (q - np.mean(q)) / (np.std(q))  
        c = np.correlate(p, q, 'full')
        return c

    ccf_ielts = ccf_values(x, y)

    lags = signal.correlation_lags(len(x), len(y))

    def ccf_plot(lags, ccf):
        fig, ax =plt.subplots(figsize=(9, 6))
        ax.plot(lags, ccf, color = "black")
        ax.axhline(-2/np.sqrt(23), color='tab:red', label='5% confidence interval')
        ax.axhline(2/np.sqrt(23), color='tab:red')
        ax.axvline(x = 0, color = 'black', lw = 1)
        ax.axhline(y = 0, color = 'black', lw = 1)
        ax.axhline(y = np.max(ccf), color = 'tab:blue', lw = 1, 
        linestyle='--', label = 'highest +/- correlation')
        ax.axhline(y = np.min(ccf), color = 'tab:blue', lw = 1, 
        linestyle='--')
        ax.set(ylim = [-1, 1])
        ax.set_ylabel('Correlation Coefficients', fontsize = 12)
        ax.set_xlabel('Time Lags', fontsize = 12)
        plt.title("x leads y          |          y leads x")
        plt.legend()

    print("lag_max: ", lags[np.argmax(ccf_ielts)])
    print("lag_min: ", lags[np.argmin(ccf_ielts)])
    ccf_plot(lags, ccf_ielts)

In [ ]:
def high_pass(data, order):
    
    cut_off_frequency = 1.0 / (8/dT) #convert periode of 8000 to frequency to remove milankovic

    # Erzeugen des High-Pass-Filters
    b, a = butter(order, cut_off_frequency, btype='high', analog=False)

    # Anwenden des Filters auf die Zeitreihe
    return lfilter(b, a, data)

In [ ]:
def band_pass(data, lowcut, highcut, order, dT):
    # Konvertiere Perioden zu Frequenzen
    lowcut_frequency = 1.0 / (lowcut / dT)
    highcut_frequency = 1.0 / (highcut / dT) 

    # Erzeuge die Koeffizienten für den Bandpass-Filter
    b, a = butter(order, [lowcut_frequency, highcut_frequency], btype='band', analog=False)

    # Anwenden des Filters auf die Zeitreihe
    return lfilter(b, a, data)

In [ ]:
def moving_average(data, window_size):
    cumsum = np.cumsum(data, dtype=float)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

In [ ]:
def moving_midpoint(data, window_size):
    midpoint_values = []

    for i in range(len(data) - window_size + 1):
        window_data = data[i:i + window_size]

        min_value = np.min(window_data)
        max_value = np.max(window_data)

        midpoint = (min_value + max_value) / 2
        midpoint_values.append(midpoint)

    return np.array(midpoint_values)

In [ ]:
def smooth_moving_midpoint(data, window_size, window_size_smooth):
    laufender_mittelpunkt = moving_midpoint(data, window_size)
    smoothed_midpoint = np.convolve(laufender_mittelpunkt, np.ones(window_size_smooth)/window_size_smooth, mode='valid')
    
    return smoothed_midpoint

In [ ]:
def subtract_moving(data, window_size, mode):
    if mode == "avg":
        moveavg = data[int(window_size/2):-int(window_size/2-1)]-moving_average(data, window_size)
        return np.pad(moveavg, (int(window_size/2), int(window_size/2-1)), mode='constant', constant_values=0)

    if mode == "mid":
        movemid = data[int(window_size/2):-int(window_size/2-1)]-moving_midpoint(data, window_size)
        return np.pad(movemid, (int(window_size/2), int(window_size/2-1)), mode='constant', constant_values=0)

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order)
    y = lfilter(b, a, data)
    return y

def return_band_pass(para):
    lowcut = para[0] / (2*np.pi)
    highcut = para[1] / (2*np.pi)
    sampling_rate = 1 / 0.02  
    if lowcut < highcut:
        return butter_bandpass_filter(series_forced, lowcut, highcut, sampling_rate, 3)
    else:
        return np.zeros(len(series_forced))

def cost_function(params, y_a):
    y_b = return_band_pass(params)
    #return np.sum((y_a[bandpass_cut:-bandpass_cut] - y_b[bandpass_cut:-bandpass_cut])**2)
    return np.sum((y_a[bandpass_cut:] - y_b[bandpass_cut:])**2)

In [ ]:
def seasaw(Tn, tau, Ts0):
    # Create an interpolation function for Tn
    tn_interp = interp1d(np.arange(len(Tn)), Tn, kind='linear')
    t_values = np.arange(len(Tn))

    # Initialize the time series Ts with zeros
    Ts = np.zeros(len(t_values))

    # Perform integration for each time step
    for i in range(len(t_values)):
        t = t_values[i]
        integral, _ = quad(lambda t_prime: tn_interp(t - t_prime) * np.exp(-t_prime / tau), 0, t, limit=1000)
        Ts[i] = -1 / tau * integral + Ts0 * np.exp(-t / tau)

    return Ts

In [ ]:
def biplot(score, coeff , y):
    '''
    Author: Serafeim Loukas, serafeim.loukas@epfl.ch
    Inputs:
       score: the projected data
       coeff: the eigenvectors (PCs)
       y: the class labels
   '''    
    xs = score[:,0] # projection on PC1
    ys = score[:,1] # projection on PC2
    n = coeff.shape[0] # number of variables
    plt.figure(figsize=(10,8), dpi=100)
    classes = np.unique(y)
    colors = ['g','r','y']
    markers=['o','^','x']
    for s,l in enumerate(classes):
        plt.scatter(xs[y==l],ys[y==l], c = colors[s], marker=markers[s]) # color based on group
    for i in range(n):
        #plot as arrows the variable scores (each variable has a score for PC1 and one for PC2)
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], color = 'k', alpha = 0.9,linestyle = '-',linewidth = 1.5, overhang=0.2)
        plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'k', ha = 'center', va = 'center')

    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    limx= int(xs.max()) + 1
    limy= int(ys.max()) + 1
    plt.xlim([-limx,limx])
    plt.ylim([-limy,limy])
    plt.grid()
    plt.tick_params(axis='both', which='both')

In [ ]:
def PCA_(data1, data2, comp, i):
    combined_data = np.vstack((data1, data2)).T
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(combined_data)
    
    if i == 1:
        # Überprüfen, ob das Vorzeichen der Hauptkomponente geändert werden muss
        if np.corrcoef(pca_result[:, comp], data1)[0, 1] < 0:
            return -pca_result[:, comp]
        else:
            return pca_result[:, comp]
    else:
        return pca_result[:, comp]

In [ ]:
def PCA_(data1, data2, comp, ref_direction=None):
    combined_data = np.vstack((data1, data2)).T
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(combined_data)
    
    # Check if reference direction is provided and flip PCs if needed
    if ref_direction is not None:
        if pca.components_[0][0] * ref_direction[0][0] < 0:
            pca_result[:, 0] *= -1
        if pca.components_[1][1] * ref_direction[1][1] < 0:
            pca_result[:, 1] *= -1
   
    return [pca_result[:, comp], pca.components_]

# TEST DATA

Funktion: $ax^4 + bx^3 + cx^2 + dx$

Parameter input: $a = [d,c,b,a, N_{obs}, N_{dyn}]$

In [ ]:
t = np.arange(-108.3, -10.4, dT)
t_ = [x for x in t]

df = pd.read_table(path+"insolation_mean_mon_7_65N_100k.txt", sep = "     ", header = None, engine = "python")
df.columns = ["Time", "Insolation"]
df["Time"] = df["Time"]-0.05# + 2.000
max_val, min_val = np.round(np.max(df["Time"]),3), np.round(np.min(df["Time"]),3)
max_difference = np.round(df['Time'][df["Time"]>t_[0]].diff().abs().max(),3)
soln = interp1d(df["Time"], df["Insolation"], kind="linear")(t_)
soln_ = soln
soln = (soln - np.mean(soln)) / np.std(soln)

In [ ]:
dT = 0.02
h = 0.001
L = int(dT/h)
t = np.arange(-97.9, 0, dT)
n = len(t) 

z = 0.7
#   * , höhe barriere (negativ) , * ,  steilheit potential (positiv), messrauschen, dynamisches rauschen
a = [0.1, -1.6,  0.8, 1, 0.1, 1.5]
a = [0.1, -1,  0.8, 1, 0.1, 1.5]
#a = [0.1, -1, -0.8, 1, 0.1, 1.5]
#a = [0, -1, 0, 1, 0.1, 1.5]

### Time integration of the SDE by Euler-Maruyama method ### 
p = a        

forcing = 0.6*soln

ys=[]
for i in range(4,5):
    y = [0] * n  
    random.seed(i)                     
    y[0] = z + random.gauss(0, p[4])    
    for k in range(1, n):
        for l in range(int(L)):
            z = z - h*(4 * p[3]*z**3 + 3*p[2]*z**2 + 2*p[1]*z + p[0]) + random.gauss(0, p[5] * np.sqrt(h))
        y[k] = z + random.gauss(0, p[4]) # observation at every L(=20) years
    y += forcing #add periodic forcing
    ys.append(y)
    
fig, ax = plt.subplots(1, 2, figsize=(12,5))
xx = np.arange(-3, 3, 0.005)
ax[0].plot(xx, [potential(x) for x in xx], label="true")
ax[0].set_ylim(-2,1)
ax[0].set_xlim(-2,2)
ax[1].plot(t, y)
plt.show()

In [ ]:
n_samples = 2
# operate bootstrap
np.random.seed(5)

bts = BootstrappingWrapper(ConvolutionSmoother(window_len=20, window_type='bartlett'), 
                           bootstrap_type='mbb', block_length=40)
bts_samples = bts.sample(y, n_samples=n_samples)

# plot the bootstrapped timeseries
plt.figure(figsize=(13,5))
plt.plot(t,y)
for i in range(1, n_samples):
    plt.plot(t, bts_samples.T[:,i] + 3*i)
    ys.append(list(bts_samples.T[:,i]))
plt.show()

In [ ]:
suspected_frequency = 0.3
bandpass_cut = int(1/dT)

series_forced = ys[0]

lo = np.array([0.000001, suspected_frequency-0.001])
up = np.array([suspected_frequency+0.001, suspected_frequency+1])

ensemble = 100
set = lhs(len(lo), samples=ensemble)                
set = lo + set.dot(np.diag(up - lo))
    
for i in range(1):
    #print("Lauf:",i+1)
    fit = minimize(cost_function, set[0], args=(forcing), method="L-BFGS-B", bounds=list(zip(lo, up)))
    #print("Parameter:", fit.x, fit.fun)
    value = fit.x

for i in range(1,ensemble):
    #print("Lauf:",i+1)
    fit_ = minimize(cost_function, set[i], args=(forcing), method="L-BFGS-B", bounds=list(zip(lo, up)))
    #print("Parameter:", fit_.x, fit_.fun)
    if np.abs(fit_.fun) < np.abs(fit.fun):
        fit = fit_

band_pass_optimized = return_band_pass(fit.x)

plt.plot(t, forcing, label='true forcing')
plt.plot(t, band_pass_optimized, "--", c="tab:green", label='band-pass filtered')
#plt.plot(t[bandpass_cut:-bandpass_cut], band_pass_optimized[bandpass_cut:-bandpass_cut], c = "tab:green", label='optimization part')
plt.plot(t[bandpass_cut:], band_pass_optimized[bandpass_cut:], c = "tab:green", label='optimization part')


plt.legend()
plt.show()

print("Optimierte Parameter:", fit.x, fit.fun)

In [ ]:
model1 = np.poly1d(np.polyfit(t, soln, 16))
plt.plot(t, ngrip-np.mean(ngrip), label='Daten')
plt.plot(t, model1(t)-np.mean(model1(t)), label = "fit")
series_forced = cal
plt.plot(t, 3*cal - return_band_pass([0.17646785, 0.37686754]))
plt.plot(t, soln, label = "sol")
plt.legend()
plt.show()

In [ ]:
emb_dim = int(2*np.pi/0.25/dT)

#y_SSA1 = SSA(y,emb_dim,1,0,1)
#y_SSA2 = SSA(y,emb_dim,1,1,2)
#y_SSA3 = SSA(y,emb_dim,1,2,3)

#df = y_SSA1 + y_SSA2
#df.to_csv('y_SSA1und2.csv', index=False)

In [ ]:
import pywt

wavelet = 'db1'  
coeffs = pywt.wavedec(y, wavelet)
threshold = 1
coeffs[1:] = (pywt.threshold(c, threshold, mode="hard") for c in coeffs[1:])
denoised_values = pywt.waverec(coeffs, wavelet)
y_denoised = denoised_values[:-1]

plt.plot(t,y)
plt.plot(t,y_denoised)
series_forced = y
plt.plot(t, return_band_pass([0.02,0.5]))
plt.plot(t,forcing)

In [ ]:
window_size = 516
plt.plot(t, forcing)
plt.plot(t[int(window_size/2):-int(window_size/2-1)], moving_midpoint(y, window_size))

In [ ]:
def to_minimize(window):
    return rms_error(y-forcing, subtract_moving(y, window, "mid"))

def rms_error(x_data, y_data):
    return np.sqrt(np.mean((x_data - y_data)**2))

window_sizes = np.arange(20,800,2)

best_guess = 10

for i in window_sizes:
    result = to_minimize(i) 
    if result < to_minimize(best_guess):
        best_guess = i
        
print(best_guess, to_minimize(best_guess))

plt.plot(y-forcing)
plt.plot(subtract_moving(y, best_guess, "mid"))
plt.show()

In [ ]:
def to_minimize(lag):
    return rms_error(y_denoised-forcing+lag, subtract_moving(y, 236, "mid"))

result = minimize(to_minimize, 5)
print(result.x)

In [ ]:
#best moving mid: 516 0.3964177872245981
#best moving avg: 546 0.4879361068120511

In [ ]:
ensemble = 1
all_params = []

for i in range(len(ys)):
    
    data = list(subtract_moving(y, 550, "mid"))
    data = (data-np.mean(data))/np.std(data)
    data = data_cut(data,-60,-25)[1]
    
    ukf_1dim.mode = "no"
    ukf_1dim.hem = "north"

    lo = np.array([-3,-3,-3,0,0.000001,0])
    up = np.array([3,3,3,3,3,3])

    set = lhs(len(lo), samples=ensemble)                
    set = lo + set.dot(np.diag(up - lo))

    fit = minimize(ukf_1dim, [1,1,1,1,1,1],  method="L-BFGS-B", bounds=list(zip(lo, up)))
    
    if all(lo[i] < fit.x[i] < up[i] for i in range(len(fit.x))):
        print("Added Parameter: ",list(fit.x), "Fun: ", fit.fun)
        all_params.append(fit.x)
        plt.plot(xx, [potential_fit(x) for x in xx], label=str(i))
        
    else:
        print("NOT added Parameter: ",list(fit.x), "Fun: ", fit.fun)
    
    
average_params = np.mean(all_params, axis=0)

plt.plot(xx, [plot_potential(x,average_params) for x in xx], c = "k", label = "mean")   
plt.plot(xx, [plot_potential(x,a) for x in xx], "--", c = "k", label="true")

plt.ylim(-2,2)
plt.xlim(-2,2)

plt.legend()
plt.show()

In [ ]:
output = list(fit.x)

aic = 2*(len(set[0]) + fit.fun)
bic = 2*fit.fun + len(set[0])*np.log(len(data))

output.append(fit.fun)
output.append(aic)
output.append(bic)

print(output)

NOTE: Asymmetrisches Potential: y-forcing & subtract_moving sind verschoben. Das ganze Problem ist bei symmetrischen Potentialen nicht? Wenn vor UKF y-forcing und subract_moving normiert werden, gibt es das Problem nicht mehr. Aber dann kann nicht das richtige Potential mit geplottet werden.

In [ ]:
plot_std(fit.x)
plt.plot(xx, plot_potential(xx, fit.x))
plt.ylim(-4,2)

In [ ]:
plt.hist(ys[0], bins = 100, alpha = 0.5, color = "k")
#plt.hist(ys[1], bins = 100)
plt.hist(ys[1], bins = 100, alpha = 0.5, color ="k")

plt.show()

nlags = 200
autocorr = stattools.acf(ys[0], nlags=nlags)
plt.plot(np.linspace(0,5,nlags+1), autocorr, c = "tab:green")
autocorr = stattools.acf(ys[1], nlags=nlags)
plt.plot(np.linspace(0,5,nlags+1), autocorr, c = "tab:red")
plt.show()

crosscorr = signal.correlate(ys[0],ys[1])
plt.plot(np.arange(-len(ys[0]) + 1, len(ys[0])), crosscorr, c="tab:blue")
crosscorr = signal.correlate(ys[0],ys[1])
plt.plot(np.arange(-len(ys[0]) + 1, len(ys[0])), crosscorr, c="tab:red")

plt.show()

In [ ]:
fit.x = [ 0.44909434,-1.23368723,  0.67289993,  0.88969982,  0.1174811,   1.43134424]
ukf_1dim.mode = "no"
Hfun = ndt.Hessian(ukf_1dim, full_output=True)
hessian_ndt  = Hfun(fit.x)
dx = np.sqrt(np.diag(np.linalg.inv(hessian_ndt[0])))
print(dx)

In [ ]:
fit.x = test1
plt.plot(xx, plot_potential(xx, fit.x))
plot_std(dys0, "tab:blue")

fit.x = test2
plt.plot(xx, plot_potential(xx, fit.x))
plot_std(dys1, "tab:orange")

fit.x = np.mean([test1, test2], axis=0)
plt.plot(xx, plot_potential(xx, fit.x))
plot_std([0.31481489, 0.20881886, 0.17195082, 0.09419389, 0.0044344 ,0.03474856], "tab:green")

plt.plot(xx, plot_potential(xx, [0.1, -1.6,  0.8, 1, 0.1, 1.5]))

plt.ylim(-2,2)

In [ ]:
plt.plot(xx, [potential_fit(x) for x in xx], label="simulated")
plt.plot(xx, [plot_potential(x,a) for x in xx], label="true")
plot_std(dx, 30)
plt.xlim(-3,3)

plt.ylim(-2,2)
plt.legend()
plt.show()

In [ ]:
p = fit.x
w = np.zeros(n)
z = y[0]
n = len(data)

random.seed(2)
w[0] = z + random.gauss(0, p[4]) 
for k in range(1, n):
    for l in range(int(L)):
        z = z - h*(4 * p[3]*z**3 + 3*p[2]*z**2 + 2*p[1]*z + p[0]) + random.gauss(0, p[5] * np.sqrt(h))
    w[k] = z + random.gauss(0, p[4]) # observation at every L(=20) years

### plotting two time series
plt.plot(t, y, label="given")
plt.plot(t, w, label="simulated")
plt.legend()
plt.show()

In [ ]:
ensemble_auto = []
for i in range(50):
    w[0] = z + random.gauss(0, p[4]) 
    for k in range(1, n):
        for l in range(int(L)):
            z = z - h*(4 * p[3]*z**3 + 3*p[2]*z**2 + 2*p[1]*z + p[0]) + random.gauss(0, p[5] * np.sqrt(h))
        w[k] = z + random.gauss(0, p[4]) 
    ensemble_auto.append(stattools.acf(w, nlags=nlags))

In [ ]:
x = y #data to choose
nlags = int(5/0.02)

autocorr = stattools.acf(x, nlags=nlags)
plt.plot(np.linspace(0,5,251), autocorr, c = "tab:green")

autocorr = stattools.acf(w, nlags=nlags)
ensemble_auto_mean = np.mean(ensemble_auto, axis=0)

std_deviation = np.std(ensemble_auto, axis=0)
plt.plot(np.linspace(0,5,251), ensemble_auto_mean, c = "tab:blue")
plt.plot(np.linspace(0,5,251), ensemble_auto_mean-std_deviation, "--", c = "tab:blue")
plt.plot(np.linspace(0,5,251), ensemble_auto_mean+std_deviation, "--", c = "tab:blue")

plt.plot(np.linspace(0,5,251), [0]*(nlags+1), "--", c = "k")
plt.show()

# REAL DATA

In [ ]:
dT = 0.02
#h = 0.001
#L = int(dT/h)
#t = np.arange(-108.3, -10.4, dT) 
t = np.arange(-67.73, -10.4, dT)
t_ = [x for x in t]

n = len(t) 
y = [0] * n  

In [ ]:
# Import real data:
path = "C:/Users/morit/documents/Masterarbeit/data/"

### NGRIP d18O 20 year resolution ###
df = pd.read_csv(path + "NGRIP_d18O_GICC05modelext.txt", sep = "\t", header = 66, engine = "python")
df["Age (years b2k)"] = -df["Age (years b2k)"]/1000 # convert to kyr
df["Age (years b2k)"] = df["Age (years b2k)"]-0.05 # convert b2k to BP
df["Age (years b2k)"] = df["Age (years b2k)"]-0.01 # datapoint reflects midpoint of interval
df["Age (years b2k)"] = df["Age (years b2k)"]*1.0063 # For consistency with WD2014 chronology and Hulu cave 
max_val, min_val = np.round(np.max(df["Age (years b2k)"]),3), np.round(np.min(df["Age (years b2k)"]),3)
max_difference = np.round(df['Age (years b2k)'][df['Age (years b2k)']>t_[0]].diff().abs().max(),3)
print("NGRIP1:  oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
ngrip = interp1d(df["Age (years b2k)"], df["NGRIP d18O (permil)"], kind="linear")(t_)
ngrip_=ngrip
ngrip = (ngrip - np.mean(ngrip)) / np.std(ngrip)
#plt.plot(t,ngrip, label = r"ngrip1 d18O")


### NGRIP d18O 20 year resolution version 2 ###
df = pd.read_csv(path + "ngrip_20yr.txt", sep = "\t", header = 50, engine = "python")
df_ = pd.DataFrame(df['Age'])
df_['d18O'] = df['d18O NGRIP1'].combine_first(df['d18O NGRIP2'])
df_['d18O'] = df_['d18O'].str.replace(',', '.')
df_ = df_.drop(df_.index[0])
df_['Age']=pd.to_numeric(df_['Age'])
df_["Age"] = -df_["Age"]/1000 # convert to kyr
df_["Age"] = df_["Age"]-0.05 # convert b2k to BP
df_["Age"] = df_["Age"]-0.01
df_["Age"] = df_["Age"]*1.0063 # For consistency with WD2014 chronology and Hulu cave 
max_val, min_val = np.round(np.max(df_["Age"]),3), np.round(np.min(df_["Age"]),3)
max_difference = np.round(df_['Age'][df_['Age']>t_[0]].diff().abs().max(),3)
print("NGRIP2:  oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
ngrip2 = interp1d(df_["Age"], df_["d18O"], kind="linear")(t_)
ngrip2_=ngrip2
ngrip2 = (ngrip2 - np.mean(ngrip2)) / np.std(ngrip2)
#plt.plot(t,ngrip2, label = r"ngrip2 d18O")


### NGRIP calcium 20 year resolution ###
df = pd.read_csv(path + "ngrip_20yr.txt", sep = "\t", header = 50, engine = "python")
df_ = pd.DataFrame(df['Age'])
df_['Ca2+'] = df['[Ca2+] NGRIP2']
df_['Ca2+'] = df_['Ca2+'].str.replace(',', '.')
df_ = df_.drop(df_.index[0])
df_['Age']=pd.to_numeric(df_['Age'])
df_["Age"] = -df_["Age"]/1000
df_["Age"] = df_["Age"]-0.05
df_["Age"] = df_["Age"]-0.01
df_["Age"] = df_["Age"]*1.0063
df_cleaned = df_.dropna(subset=['Ca2+']) 
max_val, min_val = np.round(np.max(df_["Age"]),3), np.round(np.min(df_["Age"]),3)
max_difference = np.round(df_['Age'][df_["Age"]>t_[0]].diff().abs().max(),3)
print("CALCIUM: oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
cal = interp1d(df_cleaned['Age'].values, df_cleaned['Ca2+'].values, kind="linear")(t_)
cal = -np.log(cal)
cal_= cal
cal = (cal - np.mean(cal)) / np.std(cal)
#plt.plot(t,cal, label = r"ngrip calcium")

### insolation north ###
#http://vo.imcce.fr/insola/earth/online/earth/online/index.php
df = pd.read_table(path+"insolation_mean_mon_7_65N_100k.txt", sep = "     ", header = None, engine = "python")
df.columns = ["Time", "Insolation"]
df["Time"] = df["Time"]-0.05# + 2.000
max_val, min_val = np.round(np.max(df["Time"]),3), np.round(np.min(df["Time"]),3)
max_difference = np.round(df['Time'][df["Time"]>t_[0]].diff().abs().max(),3)
soln = interp1d(df["Time"], df["Insolation"], kind="linear")(t_)
soln_ = soln
soln = (soln - np.mean(soln)) / np.std(soln)
print("INSO (N): oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
#plt.plot(t,soln, label = "sol (n)")

### insolation south ###
df = pd.read_table(path+"insolation_mean_mon_7_65S_100k.txt", sep = "     ", header = None, engine = "python")
df.columns = ["Time", "Insolation"]
df["Time"] = df["Time"] + 2.000
max_val, min_val = np.round(np.max(df["Time"]),3), np.round(np.min(df["Time"]),3)
max_difference = np.round(df['Time'][df["Time"]>t_[0]].diff().abs().max(),3)
sols = interp1d(df["Time"], df["Insolation"], kind="linear")(t_)
sols_ = sols
sols = (sols - np.mean(sols)) / np.std(sols)
print("INSO (S):  oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
#plt.plot(t,sols, label = "sol (s)")


### benthic ###
df = pd.read_csv(path  + "Global_stack_d18O.txt", sep = "\t", header = 57, engine = "python")
df["Age [ka BP]"] = -df["Age [ka BP]"]# + 1.950
max_val, min_val = np.round(np.max(df["Age [ka BP]"]),3), np.round(np.min(df["Age [ka BP]"]),3)
max_difference = np.round(df['Age [ka BP]'][df["Age [ka BP]"]>t_[0]].diff().abs().max(),3)
benthic = interp1d(df["Age [ka BP]"], df["δ18O stack [‰]"], kind="linear")(t_)
benthic_= benthic
benthic = (benthic - np.mean(benthic)) / np.std(benthic)
print("BENTHIC:  oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
#plt.plot(t, -benthic, label = "benthic")


### WD d18O ###
WD_age = pd.read_csv(path + "WD2014_506modAKZ291b_v4.txt", sep = "\t", header = 12, engine = "python")
df = pd.read_csv(path + "WAIS_project_members_Source_Data.txt", sep = "\t", header = 44, engine = "python")
df1 = df[["Depths Top (m)", "Depths Bottom (m)"]].mean(axis=1).to_frame()
df1.rename(columns={0: "Depth"}, inplace=True)
df2 = df[["WD2014 Age Top (ka1950.0)", "WD2014 Age Bottom (ka1950.0)"]].mean(axis=1).to_frame()
df2.rename(columns={0: "WD2014 Age"}, inplace=True)
df3 = df["Isotope Data d18O (per mil)"]
df_ = pd.concat([df1, df2, df3], axis=1)
WD_age["Ice age (years BP)"] = -WD_age["Ice age (years BP)"]/1000
df_["WD2014 Age"] = interp1d(WD_age["Depth (m)"], WD_age["Ice age (years BP)"], kind='linear', fill_value='extrapolate')(df_["Depth"])
max_val, min_val = np.round(np.max(df_["WD2014 Age"]),3), np.round(np.min(df_["WD2014 Age"]),3)
max_difference = np.round(df_["WD2014 Age"][df_["WD2014 Age"]>t_[0]].diff().abs().max(),3)
print("WD:        oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
wd = interp1d(df_["WD2014 Age"], df_["Isotope Data d18O (per mil)"], kind="linear")(t_)
wd_= wd
wd = (wd - np.mean(wd)) / np.std(wd)
#plt.plot(t,wd, label = r"WD")


### EDML d18O ###
df = pd.read_csv(path + "EDML.txt", sep = "\t", header = 22, engine = "python")
df["Age [ka BP]"] = -df["Age [ka BP]"]
max_val, min_val = np.round(np.max(df["Age [ka BP]"]),3), np.round(np.min(df["Age [ka BP]"]),3)
max_difference = np.round(df['Age [ka BP]'][df["Age [ka BP]"]>t_[0]].diff().abs().max(),3)
edml = interp1d(df["Age [ka BP]"], df["δ18O H2O [‰ SMOW]"], kind="linear")(t_)
edml_= edml
edml = (edml - np.mean(edml)) / np.std(edml)
print("EDML:    oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
#plt.plot(t,edml, label = "EDML")

### WD-EDC-EDML d18O stack ###
df = pd.read_csv(path + "antarctica_stack.txt", sep = "\t", skipinitialspace=True, header = 26, engine = "python")
df["Age [ka BP]"] = -df["Age [ka BP]"]#+1.950
max_val, min_val = np.round(np.max(df["Age [ka BP]"]),3), np.round(np.min(df["Age [ka BP]"]),3)
max_difference = np.round(df['Age [ka BP]'][df["Age [ka BP]"]>t_[0]].diff().abs().max(),3)
print("ANT_ST:   oldest: " + str(min_val) + " | " + "newest: " + str(max_val) + " | " + "largest deltaT: " + str(max_difference))
ant_st = interp1d(df["Age [ka BP]"], df["δ18O H2O [‰ SMOW]"], kind="linear")(t_)
ant_st_= ant_st
ant_st = (ant_st - np.mean(ant_st)) / np.std(ant_st)
#plt.plot(t, ant_st, label = "WD-EDC-EDML")


plt.legend()
plt.xlabel("WD2014 age (kyr BP)")
plt.show()

In [ ]:
def add_red_noise(data, gauß, alpha, seed=1):
        rng = np.random.default_rng(seed)
        noise = rng.normal(0, gauß, len(data))
        #fft = np.fft.fft(noise)
        #frequencies = np.fft.fftfreq(len(data))
        #filter = np.power(np.abs(frequencies) + 1e-10, -alpha / 2)
        #filtered_fft = fft * filter
        #filtered_noise = np.real(np.fft.ifft(filtered_fft))
        return data + noise #filtered_noise

In [ ]:
north_true = subtract_moving(cal_, 516, "mid")
south_true = subtract_moving(wd_, 516, "mid")

num_steps = len(north_true)

def correlation(params):
    
    S = np.zeros(num_steps)
    N = np.zeros(num_steps)

    S[0] = north_true[0]
    N[0] = north_true[0]

    for i in range(1, num_steps):
        S[i] = S[i-1] - (dT/params[0]) * (N[i-1] + S[i-1])
        N[i] = north_true[i]

    Tsp = add_red_noise(S, params[1], 1)
    
    return -np.corrcoef(Tsp, south_true)[0, 1]

In [ ]:
taus = np.arange(0.5,2, 0.005)
corr = []

lo = np.array([taus.min(), 0.2])
up = np.array([taus.max(), 1])

corr_min = minimize(correlation, [0.01, 0.001], method="L-BFGS-B", bounds=list(zip(lo, up)))
print(corr_min.x)

for i in taus:
    corr.append(-1*correlation([i, corr_min.x[1]]))

plt.plot(taus, corr)

plt.scatter(corr_min.x[0], -corr_min.fun)
plt.text(corr_min.x[0], -corr_min.fun+0.002, f'({np.round(corr_min.x[0],4)}, {-np.round(corr_min.fun,4)})', ha='center', va='bottom')

plt.ylim(corr[0],-corr_min.fun+0.005)
plt.show()

In [ ]:
tau = corr_min.x[0]

S = np.zeros(num_steps)
N = np.zeros(num_steps)

S[0]  = north_true[0]
N[0]  = north_true[0]

random.seed(1)
for i in range(1, num_steps):
    S[i] = S[i-1] - (dT/tau) * (N[i-1] + S[i-1])
    N[i] = north_true[i]
    
S_ = add_red_noise(S, corr_min.x[1], 1)

plt.plot(t, (south_true-np.mean(south_true))/np.std(south_true), label = 'WDC', zorder = 1)
plt.plot(t, (S-np.mean(S))/np.std(S), label = 'Calculated from NGRIP Ca$^{2+}$', c = "tab:orange", zorder = 3)
plt.plot(t, (S_-np.mean(S_))/np.std(S_), label = 'Calculated from NGRIP Ca$^{2+}$+ white noise', c = "tab:red", zorder = 2)
plt.legend()
#plt.xlim(-40,-45)
plt.show()

In [ ]:
S = (S-np.mean(S))/np.std(S)
S = add_red_noise(S, 0.15, 1)
plt.plot(S)
plt.plot(north_true)

### Modes: 

full: $[𝑑,𝑐,𝑏,𝑎,𝑁_{𝑜𝑏𝑠},𝑁_{𝑑𝑦𝑛}, benthic, sol]$

no: $[𝑑,𝑐,𝑏,𝑎,𝑁_{𝑜𝑏𝑠},𝑁_{𝑑𝑦𝑛}]$

benthic: $[𝑑,𝑐,𝑏,𝑎,𝑁_{𝑜𝑏𝑠},𝑁_{𝑑𝑦𝑛}, benthic]$

sol: $[𝑑,𝑐,𝑏,𝑎,𝑁_{𝑜𝑏𝑠},𝑁_{𝑑𝑦𝑛}, sol]$

In [ ]:
#load data
data1 = north_true #cal_ 
data2 = S #wd_

#subract forcing
#data1 = list(subtract_moving(data1, 516, "mid"))
#data1_return = data1
#data2 = list(subtract_moving(data2, 516, "mid"))

#normalize
#data1 = data1-np.mean(data1)
#data2 = data2-np.mean(data2)
#data1 = (data1-np.mean(data1))/np.std(data1)
#data2 = (data2-np.mean(data2))/np.std(data2)

#cut 
#lag = (208+218)/2/1000
#data1 = data_cut(data1,-60,-25)[1]
#data2 = data_cut(data2,-60+lag,-25+lag)[1]
#lag = 23
#data1 = data_cut(data1,-60,-25)[1]
#data2 = data_cut(data2[:-lag],-60-lag*dT,-25-lag*dT)[1]
data1 = data_cut(data1,-60,-25)[1]
data2 = data_cut(data2,-60,-25)[1]

#plot PCA
plt.plot(data_cut(data1,-60,-25)[0], data1, label = "Cal")
plt.plot(data_cut(data1,-60,-25)[0], data2, label = "WDC")
plt.legend()
plt.show()

#ref_direction = PCA_(-data1, -data2, None, None)[1]

plt.plot(data_cut(data1,-60,-25)[0], PCA_(data1, data2, 0, ref_direction)[0], label = "PC 1")
plt.plot(data_cut(data1,-60,-25)[0], PCA_(data1, data2, 1, ref_direction)[0], label = "PC 2")
plt.legend()

plt.show()

#biplot(data_new[:,0:2], np.transpose(pca.components_[0:2, :]), 2)

combined_data = np.vstack((data1, data2)).T
pca = PCA(n_components=2)
pca_result = pca.fit_transform(combined_data)

#Hauptkomponenten sind Vektoren im Merkmalsraum und definieren die Richtungen mit maximaler Varianz in den Daten.
print("PCA1: " + str(np.round(pca.components_[0], 2))) #1.ste Hauptkomponente
print("PCA2: " + str(np.round(pca.components_[1], 2))) #2.te Hauptkomponente

#Diese Eigenschaft gibt an, wie viel von der Gesamtvarianz in den Daten von 
#jeder der ausgewählten Hauptkomponenten erklärt wird. Für den ersten Wert in 
#pca.explained_variance_ratio_ erhalten Sie den Anteil der Gesamtvarianz, der 
#von der ersten Hauptkomponente erklärt wird, und für den zweiten Wert erhalten 
#Sie den Anteil der Gesamtvarianz, der von der zweiten Hauptkomponente erklärt wird.
print("Varianz:   " + str(np.round(pca.explained_variance_ratio_*100,2)) + " %")

In [ ]:
fig, ax1 = plt.subplots(figsize = (5,5))

ax1.scatter(data1, data2, c = "tab:red", s = 1, label = "Data")
ax1.set_xlabel("Cal", c = "tab:red")
ax1.set_xlim(-3,3)
ax1.set_ylabel("WDC", c = "tab:red")
ax1.set_ylim(-3,3)

ax2 = ax1.twinx()
ax2.scatter(PCA_(data1, data2, 0, ref_direction)[0], PCA_(data1, data2, 1, ref_direction)[0], c = "tab:green", s = 1, label = "PCA")

coeff = np.transpose(pca.components_[0:2, :])
for i in range(2):
    ax2.arrow(0, 0, coeff[i,0], coeff[i,1], head_width=0.2, head_length=0.2, color = 'tab:green', alpha = 1, linestyle = '-', linewidth = 1.5, overhang=0.2)
    bbox_props = dict(boxstyle="round,pad=0.3", fc="white", ec="tab:green", lw=1) 
    ax2.text(coeff[i,0]* 1.4, coeff[i,1] * 1.5, "Var"+str(i+1), color = 'tab:green', ha = 'center', va = 'center', bbox=bbox_props)

ax2.set_ylabel("PC 2", c = "tab:green")
ax2.set_ylim(-3,3)

ax3 = ax1.twiny()
ax3.set_xlabel('PC 1', c = "tab:green")
ax3.set_xlim(-3,3)

plt.show()

In [ ]:
ensemble = 1
xx = np.arange(-4, 4, 0.005)
#all_params = []

for index in range(1):
    
    data = PCA_(data1, data2, 1, ref_direction)[0]
  
    ukf_1dim.mode = "no"
    ukf_1dim.hem = "north"

    bound_int = 3
    lo = np.array([-bound_int, -bound_int, -bound_int,  0, 0.000001, 0])
    up = np.array([bound_int,bound_int,bound_int,bound_int,bound_int,bound_int+2])

    set = lhs(len(lo), samples=ensemble)                
    set = lo + set.dot(np.diag(up - lo))

    fit = minimize(ukf_1dim, [1,1,1,1,1,1], method="L-BFGS-B", bounds=list(zip(lo, up)))
    
    if all(lo[index] < fit.x[index] < up[index] for index in range(len(fit.x))):
        print("Added Parameter: ", fit.x, "Fun: ", fit.fun)
        #all_params.append(fit.x)
        plt.plot(xx, [potential_fit(x) for x in xx], label="simulated")
        
    else:
        #print("NOT added Parameter: ",fit.x, "Fun: ", fit.fun)
        plt.plot(xx, [potential_fit(x) for x in xx], "--", label="simulated")
    
    
#average_params = np.mean(all_params, axis=0)

#plt.plot(xx, [plot_potential(x,average_params) for x in xx], c = "k", label = "mean")   

plt.xlim(-3,3)
plt.ylim(-2,2)

plt.legend()
plt.show()

In [ ]:
output = list(fit.x)

aic = 2*(len(set[0]) + fit.fun)
bic = 2*fit.fun + len(set[0])*np.log(len(data))

output.append(fit.fun)
output.append(aic)
output.append(bic)

print(output)

In [ ]:
data = PCA_(data1, data2, 0, ref_direction)[0]
ukf_1dim.mode = "no"
ukf_1dim.hem = "north"
fit_x = fit.x
Hfun = ndt.Hessian(ukf_1dim, full_output=True)
hessian_ndt  = Hfun(fit_x)
dx = list(np.sqrt(np.diag(np.linalg.inv(hessian_ndt[0]))))
print(dx)

In [ ]:
plot_std(dx, 60)
#plot_std_(dx)
plt.plot(xx, plot_potential(xx, fit.x))
plt.ylim(-2,2)

In [ ]:
xx = np.arange(-4, 4, 0.005)

#plot_std(dx, 80)
plt.plot(xx, [potential_fit(x) for x in xx])

plt.xlim(-1.5,1.5)
plt.ylim(-3,3)

plt.title("PCA from Antarctica and Greenland after removing large \n scale trend by the first two SSA. Data cut to -70 to -20 kyr BP")
#plt.savefig('C:/Users/morit/Documents/Masterarbeit/output_M/PCA.jpg', format='jpg', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
num_samples = 1000
plot_std(dx, 70)

plt.plot(xx, [potential_fit(x) for x in xx])
plt.ylim(-1,1)
plt.xlim(xx[0],xx[-1])
plt.show()

In [ ]:
p = fit.x
w = np.zeros(n)
z = data[0]
n = len(data)

w[0] = z + random.gauss(0, p[4]) 
for k in range(1, n):
    for l in range(int(L)):
        z = z - h*(4 * p[3]*z**3 + 3*p[2]*z**2 + 2*p[1]*z + p[0]) + random.gauss(0, p[5] * np.sqrt(h))
    w[k] = z + random.gauss(0, p[4]) # observation at every L(=20) years

### plotting two time series
plt.plot(data, label="given")
plt.plot(w, label="simulated")
plt.legend()
plt.show()

In [ ]:
ensemble_auto = []
for i in range(100):
    w[0] = z + random.gauss(0, p[4]) 
    for k in range(1, n):
        for l in range(int(L)):
            z = z - h*(4 * p[3]*z**3 + 3*p[2]*z**2 + 2*p[1]*z + p[0]) + random.gauss(0, p[5] * np.sqrt(h))
        w[k] = z + random.gauss(0, p[4]) 
    ensemble_auto.append(stattools.acf(w, nlags=nlags))

In [ ]:
x = y
nlags = int(5/0.02)

autocorr = stattools.acf(x, nlags=nlags)
plt.plot(np.linspace(0,5,251), autocorr, c = "tab:green")

autocorr = stattools.acf(w, nlags=nlags)
ensemble_auto_mean = np.mean(ensemble_auto, axis=0)

std_deviation = np.std(ensemble_auto, axis=0)
plt.plot(np.linspace(0,5,251), ensemble_auto_mean, c = "tab:blue")
plt.plot(np.linspace(0,5,251), ensemble_auto_mean-std_deviation, "--", c = "tab:blue")
plt.plot(np.linspace(0,5,251), ensemble_auto_mean+std_deviation, "--", c = "tab:blue")

plt.plot(np.linspace(0,5,251), [0]*(nlags+1), "--", c = "k")
plt.show()